In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget https://archive.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz

--2022-07-24 09:56:21--  https://archive.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233333392 (223M) [application/x-gzip]
Saving to: ‘spark-2.4.7-bin-hadoop2.7.tgz’

spark-2.4.7-bin-had 100%[===================>] 222.52M  18.5MB/s    in 21s     

2022-07-24 09:56:42 (10.6 MB/s) - ‘spark-2.4.7-bin-hadoop2.7.tgz’ saved [233333392/233333392]



In [ ]:
!tar -xzf /content/spark-2.4.7-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark

In [ ]:
findspark.init("/content/spark-2.4.7-bin-hadoop2.7/")

In [ ]:
findspark.find()

'/content/spark-2.4.7-bin-hadoop2.7/'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("CorrelationExample").config('spark.jars.packages', 'com.databricks:spark-xml_2.11:0.4.1') \
        .getOrCreate()

    # .config('spark.jars.packages', 'com.databricks:spark-xml_2.11:0.6.0')\

In [ ]:
spark.sparkContext.getConf().getAll()

[('spark.driver.host', 'a9dbc39c4b08'),
 ('spark.submit.pyFiles',
  '/root/.ivy2/jars/com.databricks_spark-xml_2.11-0.4.1.jar'),
 ('spark.app.name', 'CorrelationExample'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1658656961598'),
 ('spark.jars',
  'file:///root/.ivy2/jars/com.databricks_spark-xml_2.11-0.4.1.jar'),
 ('spark.jars.packages', 'com.databricks:spark-xml_2.11:0.4.1'),
 ('spark.repl.local.jars',
  'file:///root/.ivy2/jars/com.databricks_spark-xml_2.11-0.4.1.jar'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.deployMode', 'client'),
 ('spark.files',
  'file:///root/.ivy2/jars/com.databricks_spark-xml_2.11-0.4.1.jar'),
 ('spark.driver.port', '43737'),
 ('spark.ui.showConsoleProgress', 'true')]

In [ ]:
df = spark.read \
    .format("xml") \
    .option("rowTag", "person") \
    .load("/content/drive/MyDrive/test1.xml")


In [ ]:
df.printSchema()

root
 |-- _id: long (nullable = true)
 |-- addresses: struct (nullable = true)
 |    |-- address: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- city: string (nullable = true)
 |    |    |    |-- state: string (nullable = true)
 |    |    |    |-- street: string (nullable = true)
 |-- dob_month: long (nullable = true)
 |-- dob_year: long (nullable = true)
 |-- firstname: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- salary: struct (nullable = true)
 |    |-- _VALUE: long (nullable = true)
 |    |-- _currency: string (nullable = true)



In [ ]:
df.show()

+---+--------------------+---------+--------+---------+------+--------+----------+---------------+
|_id|           addresses|dob_month|dob_year|firstname|gender|lastname|middlename|         salary|
+---+--------------------+---------+--------+---------+------+--------+----------+---------------+
|  1|[[[NewJersy, NJ, ...|        1|    1980|    James|     M|   Smith|      null|  [10000, Euro]|
|  2|[[[new york, NY, ...|        6|    1990|  Michael|     M|    null|      Rose|[10000, Dollor]|
+---+--------------------+---------+--------+---------+------+--------+----------+---------------+



In [ ]:
spark.sparkContext.addFile("/content/drive/MyDrive/test1.xsd")

In [ ]:
df_schema=spark.read.format("xml").option("rowTag","Root").\
option("rowValidationXSDPath","test1.xsd").load("/content/drive/MyDrive/test1.xml")

In [ ]:
df.printSchema()

root
 |-- _id: long (nullable = true)
 |-- addresses: struct (nullable = true)
 |    |-- address: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- city: string (nullable = true)
 |    |    |    |-- state: string (nullable = true)
 |    |    |    |-- street: string (nullable = true)
 |-- dob_month: long (nullable = true)
 |-- dob_year: long (nullable = true)
 |-- firstname: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- salary: struct (nullable = true)
 |    |-- _VALUE: long (nullable = true)
 |    |-- _currency: string (nullable = true)



In [ ]:
!/content/spark-2.4.7-bin-hadoop2.7/bin/spark-shell

22/07/24 10:29:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/24 10:30:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Spark context Web UI available at http://a9dbc39c4b08:4041
Spark context available as 'sc' (master = local[*], app id = local-1658658609917).
Spark session available as 'spark'.
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/
         
Using Scala version 2.11.12 (OpenJDK 64-Bit Server VM, Java 1.8.0_312)
Type in expressions to have them evaluated.
Type :help for more information.

scala> 

In [ ]:
spark = SparkSession \
        .builder \
        .appName("CorrelationExample").config('spark.jars.packages', 'com.databricks:spark-xml_2.11:0.4.1') \
        .getOrCreate()